### Read the case, department, and source data into their own spark dataframes.

### Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json

Inspect your folder structure. 
What do you notice?
Inspect the data in your dataframes. 
Are the data types appropriate? 

### Write the code necessary to cast the values to the appropriate types.

### How old is the latest (in terms of days past SLA) currently open issue?

### How long has the oldest (in terms of days since opened) currently opened issue been open?

### How many Stray Animal cases are there?

### How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

### Convert the council_district column to a string column.

### Extract the year from the case_closed_date column.

### Convert num_days_late from days to hours in new columns num_hours_late.

### Join the case data with the source and department data.

### Are there any cases that do not have a request source?

### What are the top 10 service request types in terms of number of requests?

### What are the top 10 service request types in terms of average days late?

### Does number of days late depend on department?

### How do number of days late depend on department and request type?